# Teste de hipótese com uma amostra (Parte 3/3)

#### Na Parte 1 mostrou-se quais são as principais distribuições de densidade de probabilidade e na Parte 2 explanou-se os tipos de intervalos de confiança apresentando os principais métodos de cada distribuição. Agora veremos como implementar funções para testes de hipótese usando um teste z, t e qui-quadrado.

#### Um teste de hipótese nada mais é do uma técnica estatística que usamos para rejeitar ou não uma hipótese. Para qualquer teste de hipótese devemos ter duas hipóteses, a hipótese nula (H0) contendo a igualdade da relação e a hipótese alternativa (Ha) que é o complemento de H0. Se uma hipótese é rejeitada a outra por exclusão necessariamente precisa ser aceita. Para decidir se rejeitamos ou não uma hipótese primeiro devemos assumir uma delas como verdadeira, e por convenção assume-se a hipótese nula. Uma vez estabelecida as duas hipótese podemos usar as estatísticas de teste para calcular um valor padronizado (estatística de teste padronizada) e um valor de área correspondende, chamado de valor p. Se o valor p encontrado é menor ou igual ao nível de significância α então devemos rejeitar a hipótese nula (aceitando Ha), caso contrário a aceitamos (rejeitando Ha).

#### Obs: As amostras devem sempre serem de populações normalmente distribuídas e/ou com tamanho de pelo menos 30 elementos (Teorema do Limite Central).

In [1]:
# Operações matemáticas e estrutura de dados.
import numpy as np

# Distribuições.
from scipy.stats import norm, t, chi2

---
## Teste z

### O teste z é usado para testar hipóteses acerca da média populacional μ (com desvio padrão populacional σ conhecido) e da proporção populacional p.

### Média Populacional com Desvio Populacional Conhecido

In [2]:
# Parâmetros:

# amostra: Amostra de dados;
# desvio: Desvio padrão populacional;
# hip: Valor da hipótese nula;
# alpha: Nível de significância;

# tipo: Tipo de teste. As opções são:
# - Unilateral à esquerda (e), quando a hipótese alternativa contém uma desigualdade com símbolo de menor;
# - Unilateral à direita (d), quando a hipótese alternativa contém uma desigualdade com símbolo de maior;
# - Bilateral (b), quando a hipótese nula é uma desigualdade com símbolo de diferente.

def teste_media_z(amostra, desvio, hip, alpha, tipo):

    # Estatística de teste padronizada usando a definição apresentada na Parte 2. 
    # Para testes de distribuições amostrais, que são nossos casos de estudo, devemos usar o erro padrão como o desvio das médias amostrais.
    z = (np.mean(amostra) - hip)/(desvio/np.sqrt(len(amostra)))

    # Módulo do valor crítico associado ao nível de confiança α. Perceba que num primeiro momento estamos assumindo que o teste é unilateral.
    z_0 = norm.isf(alpha)

    # Se o teste for unilateral à esquerda então o valor p será igual à área à esquerda da estatística de teste padronizada.
    if tipo == 'e':
        p = norm.cdf(z)

    # Se for unilateral à direita então ele será igual à área à direita.
    elif tipo == 'd':
        p = norm.sf(z)

    # Se for bilateral então o valor p será o dobro da área encontrada em alguma das caldas, nesse caso calculamos duas vezes a área à direita.
    elif tipo == 'b':
        p = norm.sf(abs(z))
        p *= 2
        # Atualiza-se o valor crítico para um teste bilateral. Cada metade do nível de confiança está em uma calda, por isso dividimos α por 2.
        z_0 = norm.isf(alpha/2)
    else:
        print('Teste inválido!')
        return None

    # Output formatado.
    print('Módulo do Valor Crítico (z0):', z_0)
    print('Estatística de Teste Padronizada (z):', z)
    print('Nível de Significância (alpha):', alpha)
    print('Valor-p:', p)
    
    print('\n----------------------------------------------\n')

    # Decisão.
    if p <= alpha:
        print('Decisão: Rejeitar Hipótese Nula')
    else:
        print('Decisão: Não Rejeitar Hipótese Nula')

    print('\n----------------------------------------------\n')

#### Vamos realizar um teste de hipótese para a média populacional μ com uma amostra de 120 elementos com média 49,5. O desvio padrão populacional σ é de 3,6, e o nível de significância α é de 5%. Devemos rejeitar ou não a hipótese que diz que μ é igual a 48,8 (teste bilateral)?

$$ \large\begin{cases}H_0: \mu = 48.8 \\ H_a: \mu \neq 48.8\end{cases} $$

In [3]:
teste_media_z([49.5] * 120, 3.6, 48.8, 0.05, 'b')

Módulo do Valor Crítico (z0): 1.9599639845400545
Estatística de Teste Padronizada (z): 2.1300321680756547
Nível de Significância (alpha): 0.05
Valor-p: 0.03316895769207681

----------------------------------------------

Decisão: Rejeitar Hipótese Nula

----------------------------------------------



#### Rejeitamos a hipótese nula. Portanto há evidência suficiente, ao nível de 95% de confiança (c = 1 - α), para concordar com a hipótese de que a média populacional é diferente de 48,8.

#### Testaremos uma hipótese para outra média populacional μ usando uma amostra de 100 elementos com:

$$ \large\begin{cases}\bar{x} = 9.9 \\ \sigma = 2.1 \\ \alpha = 0.08\end{cases} $$

#### A hipótese nula diz que μ é no máximo 10,3, devemos rejeitá-la ou não?

$$ \large\begin{cases}H_0: \mu \le 10.3 \\ H_a: \mu > 10.3\end{cases} $$

In [4]:
teste_media_z([9.9] * 100, 2.1, 10.3, 0.08, 'd')

Módulo do Valor Crítico (z0): 1.4050715603096329
Estatística de Teste Padronizada (z): -1.9047619047619062
Nível de Significância (alpha): 0.08
Valor-p: 0.9715944860326363

----------------------------------------------

Decisão: Não Rejeitar Hipótese Nula

----------------------------------------------



#### Não rejeitaremos a hipótese nula, logo podemos concordar que a média populacional é no máximo 10,3 com 92% de confiança.

### Proporção Populacional

In [5]:
# Parâmetros:

# x: Número de sucessos;
# n: Tamanho da amostra;
# hip: Valor da hipótese nula;
# alpha: Nível de significância;
# tipo: Tipo de teste.

def teste_proporcao(x, n, hip, alpha, tipo):

    # Verificando as condições que permitem uma aproximação normal da distribuição binomial.
    # Os produtos np e nq = n(1-p) devem ser maior ou igual a 5.
    if n*hip < 5 or n*(1-hip) < 5:
        print('Não se pode realizar o teste z nessa amostra pois ela não pode ser aproximada por uma distribuição normal!!')
        return None

    # Estatística de teste.
    p_est = x/n

    # Estatística de teste padronizada.
    z = (p_est - hip)/np.sqrt(hip*(1-hip)/n)

    # Módulo do valor crítico.
    z_0 = norm.isf(alpha)

    # Tipos de teste.
    if tipo == 'e':
        p = norm.cdf(z)
    elif tipo == 'd':
        p = norm.sf(z)
    elif tipo == 'b':
        p = norm.sf(abs(z))
        p *= 2
        z_0 = norm.isf(alpha/2)
    else:
        print('Teste Inválido!')
        return None

    # Output formatado.
    print('Estatística de Teste Padronizada (z):', z)
    print('Módulo do Valor Crítico (z0):', z_0)
    print('Valor-p:', p)
    print('Nível de Significância (alpha):', alpha)

    print('\n----------------------------------------------\n')

    # Decisão.
    if p <= alpha:
        print('Decisão: Rejeitar Hipótese Nula')
    else:
        print('Decisão: Não Rejeitar Hipótese Nula')

    print('\n----------------------------------------------\n')

#### Em uma amostra de 200 elementos, 39 atenderam certa condição de sucesso, podemos dizer que, ao nível de 95% de confiança, a proporcão populacional de sucessos é de pelo menos 25%?

$$ \large\begin{cases}H_0: p \ge 0.25 \\ H_a: p < 0.25\end{cases} $$

In [6]:
teste_proporcao(39, 200, 0.25, 0.05, 'e')

Estatística de Teste Padronizada (z): -1.7962924780409972
Módulo do Valor Crítico (z0): 1.6448536269514729
Valor-p: 0.03622400676489357
Nível de Significância (alpha): 0.05

----------------------------------------------

Decisão: Rejeitar Hipótese Nula

----------------------------------------------



#### Com 95% de confiança podemos rejeitar a hipótese que diz que a proporção populacional é maior ou igual a 25%.

---
## Teste t

### O teste t é usado para testar hipóteses para a média populacional μ quando não se sabe o valor do desvio padrão populacional σ.

### Média Populacional com Desvio Populacional Desconhecido

In [7]:
# Parâmetros:

# amostra: Amostra de dados;
# hip: Valor da hipótese nula;
# alpha: Nível de significância;
# tipo: Tipo de teste.

def teste_media_t(amostra, hip, alpha, tipo):

    # Desvio amostral.
    s = np.std(amostra, ddof=1)

    # Tamanho da amostra.
    n = len(amostra)

    # Graus de liberdade.
    df = n-1

    # Estatística de teste padronizada.
    t_p = (np.mean(amostra) - hip)/(s/np.sqrt(n))

    # Módulo do valor crítico.
    t_0 = t.isf(alpha, df=df)

    # Tipos de teste.
    if tipo == 'e':
        p = t.cdf(t_p, df=df)
    elif tipo == 'd':
        p = t.sf(t_p, df=df)
    elif tipo == 'b':
        p = t.sf(abs(t_p), df=df)
        p *= 2
        t_0 = t.isf(alpha/2, df=df)
    else:
        print('Teste Inválido!')
        return None

    # Output formatado.
    print('Módulo do Valor Crítico (t0):', t_0)
    print('Estatística de Teste Padronizada (t):', t_p)
    print('Nível de Significância (alpha):', alpha)
    print('Valor-p:', p)

    print('\n----------------------------------------------\n')

    # Decisão.
    if p <= alpha:
        print('Decisão: Rejeitar Hipótese Nula')
    else:
        print('Decisão: Não Rejeitar Hipótese Nula')

    print('\n----------------------------------------------\n')

In [8]:
amostra = np.array('24 36 44 35 44 34 29 40 39 43 41 32 33 29 29 43 25 39 25 42 29 22 22 25 14 15 14 29 25 27 22 24 18 17'.split(), 
                   dtype=float)
amostra

array([24., 36., 44., 35., 44., 34., 29., 40., 39., 43., 41., 32., 33.,
       29., 29., 43., 25., 39., 25., 42., 29., 22., 22., 25., 14., 15.,
       14., 29., 25., 27., 22., 24., 18., 17.])

#### Dada a amostra acima, rejeitamos ou não a hipótese de que μ é igual a 33? Usar c = 90%.

$$ \large\begin{cases}H_0: \mu = 33 \\ H_a: \mu \neq 33\end{cases} $$

In [9]:
teste_media_t(amostra, 33, 0.1, 'b')

Módulo do Valor Crítico (t0): 1.6923603138485823
Estatística de Teste Padronizada (t): -2.1146725229790633
Nível de Significância (alpha): 0.1
Valor-p: 0.04209623195922052

----------------------------------------------

Decisão: Rejeitar Hipótese Nula

----------------------------------------------



---
## Teste chi-quadrado

#### O teste qui-quadrado é usado para testar hipóteses para a variância populacional σ² e para o desvio padrão populacional σ. Diferente dos outros dois testes, na Distribuição Chi-quadrado a amostra precisa necessariamente ser de uma população normalmente distribuída, mesmo quando o tamanho da amostra é maior ou igual a 30, isso porque a Distribuição Gama, de onde se deriva a Distribuição Chi-quadrado, não é tão robusta para populações não-normais já que poucas variações são amplificadas por um fator de potência.

### Desvio e Variância Populacionais

In [10]:
# Parâmetros:

# n: Tamanho da amostra;
# s: Desvio padrão amostral;
# hip: Valor da hipótese nula;
# alpha: Nível de significância;
# tipo: Tipo de teste.


def teste_qui(n, s, hip, alpha, tipo):

    # Graus de liberdade.
    df = n-1

    # Estatística de teste padronizada usando a definição apresentada na Parte 2.
    qui = df*s**2/hip**2

    print('Estatística de Teste Padronizada (X²):', qui)

    # Como a Distribuição Chi-quadrado é assimétrica à direita devemos calcular separadamente os valores críticos.
    if tipo == 'e':
        p = chi2.cdf(qui, df)
        x_0 = chi2.ppf(alpha, df)
        print('Valor Crítico (X0):', x_0)
    elif tipo == 'd':
        p = chi2.sf(qui, df)
        x_0 = chi2.isf(alpha, df)
        print('Valor Crítico (X0):', x_0)
    elif tipo == 'b':

        # Se a estatística de teste padronizada for maior ou igual a mediana, que é o ponto mais alto da distribuição e que divide a área 
        # em duas partes iguais, então vamos calcular o valor p usando a área à direita da estatística de teste padronizada, caso contrário 
        # usaremos a área à esquerda.
        if qui >= chi2.median(df):
            p = chi2.sf(qui, df)
        else:
            p = chi2.cdf(qui, df)
        p *= 2
        xl, xr = chi2.interval(1-alpha, df)
        print('Valores Críticos (Xl e Xr):', (xl, xr))
    else:
        print('Teste Inválido!')
        return None

    print('Valor-p:', p)
    print('Nível de Significância (alpha):', alpha)

    print('\n----------------------------------------------\n')

    # Decisão.
    if p <= alpha:
        print('Decisão: Rejeitar Hipótese Nula')
    else:
        print('Decisão: Não Rejeitar Hipótese Nula')

    print('\n----------------------------------------------\n')

#### Usando uma amostra de tamanho 22 e com desvio 33,4, podemos rejeitar a hipótese de que a variância é no máximo 1296? Assumir α = 0.10

$$ \large\begin{cases}H_0: \sigma^2 \le 1296 \\ H_a: \sigma^2 < 1296\end{cases} $$

In [11]:
teste_qui(22, 33.4**2, 1296, 0.1, 'e')

Estatística de Teste Padronizada (X²): 15.559482873228164
Valor Crítico (X0): 13.239597975395304
Valor-p: 0.2061310480969814
Nível de Significância (alpha): 0.1

----------------------------------------------

Decisão: Não Rejeitar Hipótese Nula

----------------------------------------------



#### Ao nível de 10% de significância não podemos rejeitar a hipótese de que a variância populacional é de no máximo 1296.

#### Agora testaremos uma hipótese para o desvio padrão populacional. Com uma amostra de 30 elementos e desvio igual a 12, ao nível de 7% de significância, pode-se concordar com a hipótese de que o desvio é igual a 15?

$$ \large\begin{cases}H_0: \sigma = 15 \\ H_a: \sigma \neq 15\end{cases} $$

In [12]:
teste_qui(30, 12, 15, 0.07, 'b')

Estatística de Teste Padronizada (X²): 18.56
Valores Críticos (Xl e Xr): (16.81341492198671, 44.21686758361134)
Valor-p: 0.1358972079680338
Nível de Significância (alpha): 0.07

----------------------------------------------

Decisão: Não Rejeitar Hipótese Nula

----------------------------------------------

